### Import data set and get X and y matrix for test & train

In [22]:
import bz2
import zipfile
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

#test
# import data
#df = pd.read_csv("..\\data\\all_worker_sub10.csv")
df = pd.read_csv("/Users/sora/PycharmProjects/amazon-lameness-phase2/easy_hard_question_cutoff_ML/data/cl_all_sub10.csv") #mac
# Keep only the columns with certain names
columns_to_keep = ['Median', 'direction_pct', 'question_type']
#columns_to_keep = ['Mean', 'Median', 'Mode', 'SD', 'direction_pct','complete_time_median', 'complete_time_sd', 'question_type']
df2 = df.loc[:, columns_to_keep]

# Separate features and target variable
X = df2.drop('question_type', axis=1)  # Features (all columns except "question_type")
y = df2['question_type']               # Target variable ("question_type")

# Create a new DataFrame to store the percentage of "hard" and "easy" rows
y_counts = pd.DataFrame(y.value_counts(normalize=True))
y_counts.columns = ['Percentage']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


### fit a SVM model

In [26]:
from sklearn.svm import SVC

# Create SVM model
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X_train, y_train)
# Predict for the test set
y_pred = svm.predict(X_test)
# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Compute the classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)

Accuracy: 0.7313432835820896
              precision    recall  f1-score   support

        easy       0.71      0.76      0.74        33
        hard       0.75      0.71      0.73        34

    accuracy                           0.73        67
   macro avg       0.73      0.73      0.73        67
weighted avg       0.73      0.73      0.73        67



### Feature selection

 forward feature selection using decision tree

In [24]:
# Create a list to store the selected features
selected_features = []

# Loop through the features and select the best one
for i in range(len(X.columns)):
    best_feature = None
    best_score = 0
    
    for feature in X.columns:
        # Skip the features that have already been selected
        if feature in selected_features:
            continue
        
        # Add the feature to the list of selected features
        candidate_features = selected_features + [feature]
        
        # Train a decision tree model on the selected features
        clf = DecisionTreeClassifier(random_state=42)
        clf.fit(X_train[candidate_features], y_train)
        
        # Predict the class labels for the test set
        y_pred = clf.predict(X_test[candidate_features])
        
        # Compute the F1-score for the test set
        score = f1_score(y_test, y_pred, average='weighted')
        
        # Update the best feature if necessary
        if score > best_score:
            best_feature = feature
            best_score = score
    
    # Add the best feature to the list of selected features
    selected_features.append(best_feature)
    print(f'Selected feature #{i+1}: {best_feature} (F1-score: {best_score:.3f})')

Selected feature #1: direction_pct (F1-score: 0.715)
Selected feature #2: Median (F1-score: 0.729)
